In [4]:
# Module importing
import pandas as pd

#Dataset Path
DATASET_PATH = "./data"

In [7]:
# Importing dataset

df_feb= pd.read_parquet(DATASET_PATH+'/yellow_tripdata_2020-02.parquet', engine='pyarrow')
df_aug= pd.read_parquet(DATASET_PATH+'/yellow_tripdata_2020-08.parquet', engine='pyarrow')
df_nov= pd.read_parquet(DATASET_PATH+'/yellow_tripdata_2020-11.parquet', engine='pyarrow')